In [1]:
#Nicole Zawadzki
#ASTRO400B Project
#Finding the Tidal Radius of M33 Analytically

In [2]:
#import modules
import numpy as np
from ReadFile import Read
from CenterOfMass import CenterOfMass

In [4]:
#read in COM data files for MW, M31, and M33
fMW = np.genfromtxt('Orbit_MW.txt', dtype=None, names=True)
fM31 = np.genfromtxt('Orbit_M31.txt', dtype=None, names=True)
fM33 = np.genfromtxt('Orbit_M33.txt', dtype=None, names=True)

In [13]:
def TidalRadiusPoint(satgalaxy, hostgalaxy, snap):
    #find the tidal radius analytically throughout the simulation assuming the galaxies are point masses
    #input: satallite galaxy, host galaxy, snapshots
    #return: tidal radius 
    #use inputs to reconstruct filename
    #add string of the filenumber to find value "000"
    ilbl = '000' + str(snap)
    #remove all but the last 3 digits
    ilbl = ilbl[-3:]
    satfilename = "%s_"%(satgalaxy) + ilbl + '.txt'
    hostfilename = "%s_"%(hostgalaxy) + ilbl + '.txt'

    #read in data from ReadFile for satellite and host galaxy 
    sat_time, sat_total, sat_data = Read(satfilename)
    host_time, host_total, host_data = Read(hostfilename)   
    
    #index for stellar particles
    sat_ptype = np.where(sat_data['type'] == 2)
    host_ptype = np.where(host_data['type'] == 2)
    
    #find separation between M33 and M31
    x = abs(fM31['x'] - fM33['x'])
    y = abs(fM31['y'] - fM33['y'])
    z = abs(fM31['z'] - fM33['z'])
    R = np.sqrt(x**2 + y**2 + z**2)
    
    #Masses
    M = np.sum(host_data['m'][host_ptype])
    m = np.sum(sat_data['m'][sat_ptype])

    r_t = R*(m/(2*M))**(1/3)
    
    return r_t

In [14]:
test = TidalRadiusPoint('M33','M31',0)
print test

[ 201.08455808  190.37904769  179.58062785  168.61969339  157.34846361
  145.62098097  133.8397314   122.5568158   111.71561261  101.36579995
   92.17905782   85.08002233   80.757126     79.78068125   82.62642313
   88.93208364   97.14973803  106.48195246  116.16789832  126.27281022
  136.14230827  145.59045882  154.46438327  163.00081135  171.04245613
  178.16252047  184.78628602  190.70834984  196.11275481  200.78962573
  204.67162871  208.18116966  211.0931529   213.66332793  215.74038704
  217.30561221  218.30522165  218.72673568  218.64817836  218.02536389
  217.19522163  215.45581844  213.20977933  210.3923404   206.78518564
  202.72412387  197.83125537  192.00909145  185.39220588  177.64384397
  169.0411373   159.57182897  148.29247621  135.35353117  120.32821032
  103.45920452   86.07100383   71.63750554   61.17526461   55.40861576
   55.41388815   59.82050652   67.10387396   75.11962327   83.49009402
   91.26196853   98.80225807  105.75831929  112.71815515  119.4205803
  126.3

In [ ]:
class TidalRadius:
    
    def __init__(self,galaxy,snap):
        #use inputs to reconstruct filename
        #add string of the filenumber to find value "000"
        ilbl = '000' + str(snap)
        #remove all but the last 3 digits
        ilbl = ilbl[-3:]
        self.filename = "%s_"%(galaxy) + ilbl + '.txt'

        #read in data from ReadFile
        self.time, self.total, self.data = Read(self.filename)
        
        #store position and mass data from ReadFile
        self.x = self.data['x']
        self.y = self.data['y']
        self.z = self.data['z']
        self.m = self.data['m']
       
        